## 语言模型  
gram模型的缺点：  
参数系数，参数空间过大，存储和内存吃不消  
没有考虑文本中出现过多的废话词，可以用tf-idf优化  
没有考虑久远之前出现的词语对现在的影响，可以用bigram、trigram等词袋模型解决  

### 读取数据集

In [6]:
with open('./jaychou_lyrics.txt', encoding='utf-8') as f: # win环境下加个编码encoding
    corpus_chars = f.read()
print(len(corpus_chars))
print(corpus_chars[0:40])
corpus_chars = corpus_chars.replace('\n',' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:10000]

63281
想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天每天每


### 建立字符索引

In [16]:
idx_to_char = list(set(corpus_chars)) #得到所有的去重字符
char_to_idx = {char:i for i,char in enumerate(idx_to_char)} # 字典形式，每个字符对应一个唯一索引
vocab_size = len(char_to_idx)
print(vocab_size)

corpus_indices = [char_to_idx[char] for char in corpus_chars] #语料集里每个字符对应的索引集合
sample = corpus_indices[0:20]
print()
a = '-'.join([idx_to_char[idx] for idx in sample])
print(f'char: {a}')
print(f'sample: {sample}')      

1027

char: 想-要-有-直-升-机- -想-要-和-你-飞-到-宇-宙-去- -想-要-和
sample: [913, 799, 783, 724, 369, 935, 399, 913, 799, 108, 639, 468, 1006, 585, 284, 181, 399, 913, 799, 108]


In [17]:
def load_data_jay_lyrics():
    with open('./jaychou_lyrics.txt', encoding='utf-8') as f: # win环境下加个编码encoding
        corpus_chars = f.read()
    corpus_chars = corpus_chars.replace('\n',' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = {char:i for i,char in enumerate(idx_to_char)}
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

## 时序数据的采样

在训练中我们需要每次随机读取小批量样本和标签。与之前章节的实验数据不同的是，时序数据的一个样本通常包含连续的字符。假设时间步数为5，样本序列为5个字符，即“想”“要”“有”“直”“升”。该样本的标签序列为这些字符分别在训练集中的下一个字符，即“要”“有”“直”“升”“机”，即 X =“想要有直升”， Y =“要有直升机”。  

现在我们考虑序列“想要有直升机，想要和你飞到宇宙去”，如果时间步数为5，有以下可能的样本和标签：  

X ：“想要有直升”， Y ：“要有直升机”   
X ：“要有直升机”， Y ：“有直升机，”  
X ：“有直升机，”， Y ：“直升机，想”  
...  
X ：“要和你飞到”， Y ：“和你飞到宇”  
X ：“和你飞到宇”， Y ：“你飞到宇宙”  
X ：“你飞到宇宙”， Y ：“飞到宇宙去”  
可以看到，如果序列的长度为 T ，时间步数为 n ，那么一共有 T−n 个合法的样本，但是这些样本有大量的重合，我们通常采用更加高效的采样方式。我们有两种方式对时序数据进行采样，分别是随机采样和相邻采样。   

### 随机采样  
下面的代码每次从数据里随机采样一个小批量。其中批量大小batch_size是每个小批量的样本数，num_steps是每个样本所包含的时间步数。 在随机采样中，每个样本是原始序列上任意截取的一段序列，相邻的两个随机小批量在原始序列上的位置不一定相毗邻。引用下别人的图片帮助自己理解。   
![jupyter](./采样.jpg)

In [18]:
import torch
import random
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减1是因为对于长度为n的序列，X最多只有包含其中的前n - 1个字符
    num_examples = (len(corpus_indices) - 1) // num_steps  # 下取整，得到不重叠情况下的样本个数
    example_indices = [i * num_steps for i in range(num_examples)]  # 每个样本的第一个字符在corpus_indices中的下标
    random.shuffle(example_indices)

    def _data(i):
        # 返回从i开始的长为num_steps的序列
        return corpus_indices[i: i + num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(0, num_examples, batch_size):
        # 每次选出batch_size个随机样本
        batch_indices = example_indices[i: i + batch_size]  # 当前batch的各个样本的首字符的下标
        X = [_data(j) for j in batch_indices]
        Y = [_data(j + 1) for j in batch_indices]
        yield torch.tensor(X, device=device), torch.tensor(Y, device=device)

In [19]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[12, 13, 14, 15, 16, 17],
        [ 0,  1,  2,  3,  4,  5]]) 
Y: tensor([[13, 14, 15, 16, 17, 18],
        [ 1,  2,  3,  4,  5,  6]]) 

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [18, 19, 20, 21, 22, 23]]) 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [19, 20, 21, 22, 23, 24]]) 



### 相邻采样  
在相邻采样中，相邻的两个随机小批量在原始序列上的位置相毗邻。



In [32]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    corpus_len = len(corpus_indices) // batch_size * batch_size  # 保留下来的序列的长度
    corpus_indices = corpus_indices[: corpus_len]  # 仅保留前corpus_len个字符
    indices = torch.tensor(corpus_indices, device=device)
    indices = indices.view(batch_size, -1)  # resize成(batch_size, )
    batch_num = (indices.shape[1] - 1) // num_steps #计算每一块最多能取多少样本
    for i in range(batch_num):
        i = i * num_steps #避免不重复
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [33]:
a = torch.tensor([1,2,3,4,5,6,7,8,9,10,11,12])
b = a.view(3, -1)
print(b.shape)
b

torch.Size([3, 4])


tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])

In [34]:
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 0,  1,  2,  3,  4,  5],
        [15, 16, 17, 18, 19, 20]]) 
Y: tensor([[ 1,  2,  3,  4,  5,  6],
        [16, 17, 18, 19, 20, 21]]) 

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [21, 22, 23, 24, 25, 26]]) 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [22, 23, 24, 25, 26, 27]]) 

